In [5]:
import numpy as np
import torch
from torch import nn
import pickle as pkl
import torch
import wandb
wandb.init(project='M5-Image-to-text', entity='fantastic5')

from torch.optim import lr_scheduler
import torch.optim as optim

import os
import numpy as np

from trainer import fit
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib.pyplot as plt

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fantastic5 (use `wandb login --relogin` to force relogin)


In [6]:
cuda = torch.cuda.is_available()
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
class EmbeddingNet(nn.Module):
    def __init__(self, emd_dim = 4096):
        super(EmbeddingNet, self).__init__()

        self.fc1 = nn.Sequential(nn.Linear(emd_dim, 256),
                                nn.PReLU(),
                                )

    def forward(self, x1):
        output1 = self.fc1(x1)
        return output1

    def get_embedding(self, x):
        return self.forward(x)

class TripletNetAdapted(nn.Module):
    def __init__(self, image_embedding_net, word_embedding_net):
        super(TripletNetAdapted, self).__init__()
        self.image_net = image_embedding_net
        self.text_net = word_embedding_net

    def forward(self, x1, x2, x3):
        output1 = self.image_net(x1)
        output2 = self.text_net(x2)
        output3 = self.text_net(x3)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

In [7]:
def aggregation_text(word_embs, axis = 0):
    return np.sum(word_embs, axis=axis)

In [8]:
from flickrDataSet import *

# Prepare the dataset
train_data = FlickrDataset('./dataset/train_img_embs.pkl', './dataset/train_text_embs.pkl', aggregation=aggregation_text, train= True)
test_data = FlickrDataset('./dataset/test_img_embs.pkl', './dataset/test_text_embs.pkl', aggregation=aggregation_text, train= False)

triplet_train_dataset = TripletFlickrDataset(train_data) # Returns triplet of images and target same/different
triplet_test_dataset = TripletFlickrDataset(test_data)

batch_size = 64
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [24]:
a, b = train_data[2]
a

array([ 1.9144233, -0.       , -0.       , ..., -0.       , -0.       ,
       -0.       ], dtype=float32)

In [10]:
for batch_idx, (data, target) in enumerate(triplet_train_loader):
    print(batch_idx)
    break

IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\galve\Anaconda3\envs\Torch\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\galve\Anaconda3\envs\Torch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\galve\Anaconda3\envs\Torch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\galve\Master\M5\MCV-2022-M5-Project\Week5\flickrDataSet.py", line 67, in __getitem__
    img1, text_pos = self.flickr_dataset[index]
  File "c:\Users\galve\Master\M5\MCV-2022-M5-Project\Week5\flickrDataSet.py", line 36, in __getitem__
    img, text = self.train_data[index], self.img_texts[index][random.randint(0,len(self.img_texts[index]))]
IndexError: l

In [8]:
# Set up the network and training parameters
img_emb_dim = 4096
text_emb_dim = 300

save_path = 'TripletNet.pth'

embedding_net_img = EmbeddingNet(emd_dim=img_emb_dim)
embedding_net_text = EmbeddingNet(emd_dim=text_emb_dim)
model = TripletNetAdapted(embedding_net_img, embedding_net_text)


model.to(device)

if not os.path.exists(save_path):
    margin = 0.5
    loss_fn = nn.TripletMarginLoss(margin)
    lr = 1e-3
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.9, last_epoch=-1)
    n_epochs = 100
    log_interval = 1000
    ## Training !!!
    fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)
    torch.save(model.state_dict(), save_path)
else:
    print('Loading model...')
    model.load_state_dict(torch.load(save_path))
    model.to(device)

C:\Users\galve\Anaconda3\envs\Torch\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


KeyboardInterrupt: 